## LAB03a - AI Plugins and Orchestration in Semantic Kernel
***

### Part 1 - AI plugins in Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to build native and semantic functions and group them together as AI Plugins. We'll work on the following steps:

1) What are AI Plugins? 
2) Create and add Semantic Functions to a Plugin.
3) Use Semantic Kernel Tools to add a new Semantic Function
4) Create and add a Native Function to a Plugin.
5) Puting it all together for a Call Center Analytics solution

Before we begin, please ensure you have a `.env` file in your lesson directory. For this lesson, we'll also require two additional entries for Azure AI Speech and Azure AI Speech Region. Update the cell below and run it to create your `.env` file. 

In [ ]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME="" #Model Deployment name, like gpt-35-turbo-instruct
AZURE_OPENAI_ENDPOINT="" #Azure OpenAI Service endpoint, like https://cog-azopenai-demos.openai.azure.com/
AZURE_OPENAI_API_KEY="" #Azure OpenAI Service API Key
AZURE_AI_SPEECH_KEY="" #Azure Speech Service API Key
AZURE_AI_SPEECH_REGION="" #Azure Speech Service Region, like brazilsouth

#### Step 1 - What are AI Plugins?
As the Copilot concept evolves, it's important to think on ways to extend their capabilities, allowing them to retrieve external information, interact with external services and execute actions on the behavior of the user on a safer way, following Responsible AI principles.

The concept of **Plugins** is an answer to this, as they can act as a "bridge" between Copilots / AI Apps and the digital world. 
![Plugins Overview](images/plugins_overview.png)

With Plugins, it's possible to expand Copilots capabilities and promote interoperability across the industry as Semantic Kernel adopts an open standard for plugins, the [OpenAPI Specification](https://swagger.io/specification/).

Let's get started with the creation of Functions, which will be the base of our AI Plugins. In Semantic Kernel we can think of Plugins as a collection of functions. Let's see in the following sections how we can create functions and group them together as a Plugin. 

#### Step 2 - Create and add a Semantic Function to a Plugin

In the first lesson, we have created and used an [**inline**](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/inline-semantic-functions?tabs=python) Semantic Function to categorize a call.However this approach is fine for development and testing, we need a more robust way to ensure we can reuse the functions across projects and rapidly maintain / change the prompts and function parameters as required.

Therefore, we'll now explore how we can reuse such functions. Semantic Kernel allows us to import files to define functions. We must follow a standard folder structure when creating the files. Here's an example:

```
📁 plugins
│
└─── 📂 plugin_name_A
     |
     └─── 📂 function_name_A 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     └─── 📂 function_name_B 
            |
            └───📄 skprompt.txt
            └───📄 config.json
    📂 plugin_name_B
     |
     └─── 📂 function_name_C 
            |
            └───📄 skprompt.txt
            └───📄 config.json
```

Let's define a variable to represent our Plugins folder. It'll be refered later to Import the plugins.

In [ ]:
#Define where the plugins are stored. If your plugins are in a different directory, change this parameter.
plugins_directory = "./plugins"

With this structure in mind, we already start organizing our plugins and grouping together functions that belong to a plugin. For each Function (represented by a Folder) we must have two files:
* **skprompt.txt**: it's the file that will contain the prompt of a Semantic Function. Therefore, we can easily maintain our functions by reviewing the prompts;
* **config.json**: it's a configuration file that will describe the function in natural language, define which parameters the function requires, and the overall configuration the LLM should use run that function, such as temperature and maximum number of tokens. 

With this structure in mind, we already undestand how to create a folder structure to contain the funcions for a Call Center solution plugin. In this lesson, we'll recreate the function used before to fit into this structure.

To get started, run the cell below to create a new file called `skprompt.txt` file inside the Semantic Function "categorize" under "CallCenter" plugin. This file represents the Prompt our function will perform and all input parameters that we want to change in runtime, such as the problem description _{{$problem}}_ and the list with the categories the company use _{{$categories}}_.

In [ ]:
%%writefile plugins/CallCenterPlugin/categorize/skprompt.txt
You help a telco company to classify problems reported by their subscribers. Your role is to provide accurate classification based on problems description and a list of categories that the telco company uses.
Classify the problem in one of the provided categories. Only write the output category with no extra text.
Only write one category per problem description.

Examples: 
Problem: My 5G is not working well when I'm in my car.
Categories: Fixed Internet/Wifi, Mobile Internet.
Output Category: Mobile Internet

Problem: My TV is not streaming well from Youtube. It seems that the wifi in my bedroom is weak.
Categories: Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: It's impossible to work today, my internet here in my home is so slow!
Categories: Landiline, Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: {{$input}}
Categories: {{$categories}}
Output Category: 

Next, we'll review the `config.json` file to ensure it correctly describes our function. This file will define how the model should execute our prompt. Notice the parameters such as the number of tokens, temperature and description of the function and input parameters.

Run the cell below to create the file.

In [ ]:
%%writefile plugins/CallCenterPlugin/categorize/config.json
{
    "schema": 1,
    "type": "completion",
    "description": "Classify a problem report in a call to the telco company based on provided categories.",
    "completion": {
         "max_tokens": 10,
         "temperature": 0.5,
         "top_p": 0.0,
         "presence_penalty": 0.0,
         "frequency_penalty": 0.0
    },
    "input": {
         "parameters": [
              {
                   "name": "input",
                   "description": "The problem reported by the user that needs to be classified. Required parameter.",
                   "defaultValue": ""
              },
              {
                   "name": "categories",
                   "description": "A set of categories used by the telco company to classify the problems. Required parameter.",
                   "defaultValue": ""
              }
         ]
    }
}

At this moment, your Plugins folder should have _at least_ the structure below.
```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
```

To manipulate functions, we need to ensure we have all our pre-requirements set, as we did in the previous lesson. 

In [ ]:
import semantic_kernel as sk

#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion#AzureTextCompletion

# Initialize the kernel
kernel = sk.Kernel()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env() 

#We need to add least one model to the kernel
#kernel.add_text_completion_service("Text Completion", 
#                                   AzureTextCompletion(deployment, endpoint, api_key))
#Adding a model to the kernel
kernel.add_chat_service("Chat Completion",
                        AzureChatCompletion(deployment, endpoint, api_key))

Now we can refer to our Plugin and import it to be able to use its functions.

In both **config.json** and **skprompt.txt** files we have defined input parameters that "**Categorize**" function requires. In Semantic Kernel we need to use **ContextVariables** to bind values with such parameters. 

After defining the input variables, we can finally call our semantic function available in our Plugin to perform the expected operation. Please note in the code that we can have multiple functions in a Plugin, so in this case we had to explicitly mention which semantic function we would like to use.

In [ ]:
# Importing the callcenter Plugin from the plugins directory, so we can start using its functions 
callcenter_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "CallCenterPlugin"
)

#Defining the input variables for the function
var_category = sk.ContextVariables()
var_category["input"] = "My 4G is terrible slow. It's not loading even simple webpages in my mobile." #"It's impossible to work from home as my connection is so unstable. I can't even join a video call!" 
var_category["categories"] = "Landline, Mobile Internet, TV, Fixed Internet/Wifi, Billing"

#Call the plugin with input variables
category = await kernel.run_async(
    callcenter_plugin["categorize"],
    input_vars = var_category
    )

#Print the output
print(f"Category of the problem: {category}")

#### Step 3 - Use Semantic Kernel Tools to add a new Semantic Function

Another way to create Semantic Functions is to use [**Sematic Kernel Tools**](https://devblogs.microsoft.com/semantic-kernel/semantic-kernel-tools/), which facilitates the process and simplify testing of semantic functions and other tasks in Semantic Kernel.


 We'll demonstrate how to use it to create a Semantic Function. Later, you can also refer to this [tutorial](https://learn.microsoft.com/en-us/semantic-kernel/vs-code-tools/).

 Alternatively, you can run the cells below to create the "Summarize" function.

In [ ]:
%%writefile plugins/CallCenterPlugin/summarize/skprompt.txt
Summarize the problem described below. 
Be brief, yet informative.
Try to capture in the summary:
- The problem description;
- The product customer is complaining (if available);
- How agent handled the call;
- The outcome of the call. 

[INPUT]
{{$input}}
[END INPUT]

In [ ]:
%%writefile plugins/CallCenterPlugin/summarize/config.json
{
    "schema": 1,
    "type": "completion",
    "description": "Summarize a problem recorded from a call center of a telco company",
    "completion": {
        "max_tokens": 1000,
        "temperature": 0.6,
        "top_p": 0,
        "presence_penalty": 0,
        "frequency_penalty": 0
    },
    "input": {
        "parameters": [
            {
                "name": "input",
                "description": "A transcription of a call center conversation. Required parameter.",
                "defaultValue": ""
            }
        ]
    },
    "default_backends": []
}

At this point, ensure your Plugins folder structure is as follows, as we'll require both Semantic Functions later in this session.

```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     └─── 📂 summarize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
```

#### Step 4 - Create and add a Native Function to a Plugin

When we think about the concept of AI Apps it's important to remember that an application will certainly require capabilities that will not be provided solely by Large Language Models. For example, performing calculations, retrieving data from a database and even executing API calls are some of the expected capabilities an application generally has. 

Therefore, we still need to run native code and be able to combine it with AI capabilities to go beyond and create transformative applications powered by modern AI!

In semantic Kernel we can add native code in different languages, like C# and Python, to be part of our Plugins. We should use the same folder structure from before, however, instead of prompts and configurations, we will have native code, like .py (Python) or .cs (C#) files.

For example, let's assume that we want to add a new function to "CallCenter" Plugin to transcribe audio files using. Let's see how our folder structure would look like:

```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     |
     └───📄 Transcribe.py
```

##### 4.1) Transform an existing code into a Native Function for Semantic Kernel
Let's assume we already have a base code in Python that is capable of transcribing an audio file to text, as shown below:

```python
import azure.cognitiveservices.speech as speechsdk

# Define the transcribe_audio function
def transcribe_audio(audio_file, language, subscription_key, region):
    # Set up the speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    # Define the language of the audio file
    speech_config.speech_recognition_language = language

    # Open the audio file
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)
    
    #Code commited for simplicity
    "..."
    
    # Convert to string and return the transcription
    return recognized_text_list
```



 We need to do some small changes in the code to ensure it can become a native function and can be addedd to our Plugin in Semantic Kernel. In summary:
- **Transform it in a class:** all native functions must be defined as public methods that belong to a class. This class will represent our Plugin.
- **Import the required libraries:** you need to import the _skill_definition_ module to be able to use the decorators and _SKContext_ module to parse multiple input variables.
```python
#Importing Azure Cognitive Services Speech SDK package
import azure.cognitiveservices.speech as speechsdk

#Importing Semantic Kernel packages to define the function and its parameters
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)
#Importing Semantic Kernel package to parse input variables
from semantic_kernel.orchestration.sk_context import SKContext
#Defining the Transcribe class to be used as a plugin to transcribe audio files
class Transcribe:
...
```
- **Add the decorator @sk_function:** the kernel will use this decorator to be able to register this function as the plugin is loaded. The decorator will also describe the function and its input parameter (if only one), as described in the [documentation](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/native-functions/using-the-skfunction-decorator?tabs=python#use-the-skfunction-decorator-to-define-a-native-function). Such information will be later used by Planners to orchestrate Plugins.
- **Add the decorator sk_function_context_parameter:** if your function has multiple input parameters you need to use this decorator to describe the input parameters with their names and description, so the kernel understand how to call the function and orchestrate them in Planners.
```python
@sk_function(
    description="Transcribe an audio file. Suitable for long audio files",
    name="transcribe_audio",
)
@sk_function_context_parameter(
    name="audio_path",
    description="The path to the audio file to transcribe",
)
@sk_function_context_parameter(
    name="language",
    description="The language of the audio file in the format of Locale (BCP-47) (e.g. en-US)",
)
@sk_function_context_parameter(
    name="subscription_key",
    description="The subscription key for the Azure AI Speech resource",
)
@sk_function_context_parameter(
    name="region",
    description="The region of the Azure AI Speech resource",
)         
def transcribe_audio(self, context: SKContext) -> str:
```
- **Use SKContext to handle input parameters:** if your function requires multiple input parameters, you need to add _SKContext_ as the input of your function and map the individual parameters inside the function. See more details [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/native-functions/multiple-parameters?tabs=python#using-context-parameters-to-pass-multiple-inputs).
```python
    #Obtaining and parsing input parameters from SKContext
    audio_path = context["audio_path"]
    language = context["language"]
    subscription_key = context["subscription_key"]
    region = context["region"]
```

Let's review our modified code directly in the **["transcribe.py"](./plugins/CallCenterPlugin/Transcribe.py)** file in the plugins directory. If the file is not available, you can run the file below to create it.

In [ ]:
%%writefile plugins/CallCenterPlugin/Transcribe.py
import time

#Importing Azure Cognitive Services Speech SDK package
import azure.cognitiveservices.speech as speechsdk

#Importing Semantic Kernel packages to define the function and its parameters
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)
#Importing Semantic Kernel package to parse input variables
from semantic_kernel.orchestration.sk_context import SKContext

#Defining the Transcribe class to be used as a plugin to transcribe audio files
class Transcribe:
    @sk_function(
        description="Transcribe an audio file and output its text transcription.",
        name="transcribe_audio",
    )
    @sk_function_context_parameter(
        name="input",
        description="The path in disk to the audio file to transcribe. Required parameter",
    )
    @sk_function_context_parameter(
        name="language",
        description="The language of the audio file in the format of Locale (BCP-47) (e.g. en-US). Required parameter",
    )
    @sk_function_context_parameter(
        name="subscription_key",
        description="The subscription key for the Azure AI Speech resource. Required parameter.",
    )
    @sk_function_context_parameter(
        name="region",
        description="The region of the Azure AI Speech resource. Required parameter.",
    )         
    def transcribe_audio(self, context: SKContext) -> str:
        #Obtaining and parsing input parameters from SKContext
        audio_path = context["input"]
        language = context["language"]
        subscription_key = context["subscription_key"]
        region = context["region"]

        #Set up the speech configuration
        speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
        # Define the language of the audio file
        speech_config.speech_recognition_language = language

        # Open the audio file
        audio_config = speechsdk.audio.AudioConfig(filename=audio_path)

        # Create a speech recognizer
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

        # Define global variables to control the continuous recognition loop and accumulate text
        global done 
        done = False
        global recognized_text_list 
        recognized_text_list=[]

        # Define the stop callback function
        def stop_cb(evt: speechsdk.SessionEventArgs):
            """callback that signals to stop continuous recognition upon receiving an event `evt`"""
            print('CLOSING on {}'.format(evt))
            global done
            done = True

        # Define the recognize callback function
        def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
            """callback for recognizing the recognized text"""
            global recognized_text_list
            recognized_text_list.append(evt.result.text)

        # Connect callbacks to the events fired by the speech recognizer
        speech_recognizer.recognized.connect(recognize_cb)
        speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
        speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
        speech_recognizer.session_stopped.connect(stop_cb)
    
        # Start continuous speech recognition
        speech_recognizer.start_continuous_recognition()
        print("Continuous speech recognition started. Waiting to complete transcription...")
        while not done:
            time.sleep(.1)

        #Stop continuous speech recognition
        speech_recognizer.stop_continuous_recognition()
        
        print("Transcription complete.")
                
        return str(recognized_text_list)

##### 4.2) Calling a Native Function with Semantic Kernel
Now that we already have created our native function that's capable of Transcribing an audio file, let's see how can we import and use it.

The process is similar to Semantic Functions, but there's a change on how we import each of them. The native functions are imported as "Plugins" following the hierarchy and folder structure we already explored. Once we have the plugin ready as part of ou Import statement, we can use the kernel to load it using the name of the Class.

Finally, we'll define the input parameters our function requires as our Context and call the function using the kernel to run it. 

In [ ]:
#Importing the native function within the Plugin
from plugins.CallCenterPlugin.Transcribe import Transcribe

#Loading the .env file to use Azure AI Speech
import os
from dotenv import load_dotenv

#Load sensitive data from .env file to use Azure AI Speech
load_dotenv()

#Import the call center plugin to use the transcription
transcribe_fn = kernel.import_skill(Transcribe(), skill_name="CallCenterPlugin")

#Defining the input variables for the transcription funcion
var_transcribe = sk.ContextVariables()
var_transcribe["input"] = "./audio/enUS_long_network_interference.wav"#./audio/english_billing_process_sample.wav" #"./audio/enUS_short_wifiproblem.wav"
var_transcribe["language"] = "en-US"
var_transcribe["subscription_key"] = os.getenv("AZURE_AI_SPEECH_KEY")
var_transcribe["region"] = os.getenv("AZURE_AI_SPEECH_REGION")

#Call the plugin with input variables
transcription = await kernel.run_async(
    transcribe_fn["transcribe_audio"],
    input_vars = var_transcribe
    )

print(f"Transcription: {transcription.result}")

#### Step 5 - Puting it all together for a Call Center Analytics solution
Now, let's see a simple example for a Call Center Analytics solution where we:
- Use an audio as input (previous step);
- Use a Native Function to obtain the transcription from the audio (previous step);
- Use a Semantic Function to summarize the problem;
- Use a Semantic Function to category the problem;

 This is one way to chain functions together and manually orchestrate them. In another Lesson, we'll explore how to automate plugins orchestration with Planners.

In [ ]:
#Import the native functions from the Plugin
callcenter_plugin = kernel.import_semantic_skill_from_directory(plugins_directory, "CallCenterPlugin")

#Define the functions that will be used from the plugin
summarize_fn = callcenter_plugin["summarize"]
categorize_fn = callcenter_plugin["categorize"]

#Create the summary from the Transcription, running the Semantic Function
summary = summarize_fn.invoke(transcription.result)

#Define the Summary as the input for the problem categorization and run the Semantic Function
var_category["input"] = summary.result
category = categorize_fn.invoke(variables = var_category)

#Print the results
print(f"1) Transcription: {transcription.result} \n2) Summary: {summary.result} \n3)Problem Category: {category.result}")

You can also chain functions together in a sequential pipeline. In this scenario, the output of one functions acts as the input for the next one. 

In [ ]:
#Sequential pipeline to summarize and then extract the category of a problem reported in a call center conversation
result = await kernel.run_async(summarize_fn, categorize_fn, input_str=transcription.result)

print(f"Result: {result}")

***

### Part 2 - Orchestrate AI Plugins with Planners
#### Step 1 - Background

Foundation Models are revolutionizing how we as humans interact with machines, as they exceed in two important capabilities: 
- **Natural Language:** the models are capable of "understanding" how humans communicate instead of humans making adaptions to interact with machines using traditional hardware (mouse/keyboard) and UI/UX. 
- **Reasoning Engine:** the models can reason over prompts (and even videos, audio and images with multimodal) and provide outcomes from that. Traditionally, we would need to deterministically program all the application.

The planners are a great example on how we can combine both capabilities to create even richer experiences that are powered by AI. Planners are capable of transforming a high-level natural language **"ASK"** into a step-by-step **"PLAN"** (reasoning) on how to achieve that **"GOAL"**.

To do so, the Planner will try to use tools (i.e., Plugins) already loaded into the Kernel and both their native and semantic functions. Here's a high-level view on how the Planner select from available plugins to create a logical plan with the order that each step will be called.  
![Planners](https://learn.microsoft.com/en-us/semantic-kernel/media/the-planner.png)

To determine which Functions should be used and how to use it, the Planner relies on the **Description** of the functions and its parameters. Theferore, always try to describe your functions and parameters the best and clear as way as possible. Giving hints about the expected output and informing which parameters are mandatory can also improve the results when working with Planners.

Planners works as any other Semantic Function. It means that you can define your own planners writing a Prompt in a **skprompt.txt** file. We'll not explore how to create your own planner is this lab, but you can review the prompts for other planners. 

Currently, there're 4 types of out-of-the-box Planners available in Semantic Kernel for Python as described below:
| Planner | Description | Prompts |
| -------- | -------- | -------- |
|**ActionPlanner**| Creates a plan with a single step determining a single function to be called| [Link](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/action_planner/skprompt.txt) |
|**BasicPlanner** | A simplified version of SequentialPlanner that strings together a set of functions|[Link](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/basic_planner.py) |
|**SequentialPlanner**| Creates a plan with a series of steps that are interconnected with custom generated input and output variables| [Link](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)|
|**StepwisePlanner**|Incrementally performs steps and observes any results before performing the next step.|[Link](https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/stepwise_planner/Skills/StepwiseStep/skprompt.txt) |

Planners are evolving fast, so let's explore some of them and demonstrate how they work with some examples.

**Important:** As planners rely on their "reasoning" capability, generally we can obtain better results with more powerful models, like GPT-4. Please review your `.env` file below to ensure we're using this model.

In [ ]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME="" #Model Deployment name, like gpt-4
AZURE_OPENAI_ENDPOINT="" #Azure OpenAI Service endpoint, like https://cog-azopenai-demos.openai.azure.com/
AZURE_OPENAI_API_KEY="" #Azure OpenAI Service API Key
AZURE_AI_SPEECH_KEY="" #Azure Speech Service API Key
AZURE_AI_SPEECH_REGION="" #Azure Speech Service Region, like brazilsouth

#### Step 2 - Action Planner
As the solutions grow in complexity and more Plugins are developed, we may need to choose a **single** best Plugin to achieve our goal, or **ASK**. The Action Planner picks the best plugin to do what the user wants. It is different from other planners because it only runs one plugin at a time.

Let's run a example for the Action Planner. 

**Notice:** In this example, we'll import several [Core Plugins](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/out-of-the-box-plugins?tabs=python#core-plugins) (shipped with Semantic Kernel) and other plugins available as samples from [Semantic Kernel GitHub repo](https://github.com/microsoft/semantic-kernel/). You can browse an explore then later!

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

#Import the Planner from its module. Planners are part of "planning" module
from semantic_kernel.planning import ActionPlanner

# Initialize the kernel
kernel = sk.Kernel()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env() 

#Adding a model to the kernel
kernel.add_chat_service("Chat Completion",
                        AzureChatCompletion(deployment, endpoint, api_key))

#Create the Planner
planner = ActionPlanner(kernel)

#Now, we'll add the Plugins that the kernel will be allowed to use. In this case, we're importing only core Plugins 
from semantic_kernel.core_skills import MathSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(TimeSkill(), "time")

#Let's see which Plugins are available for the Planner
print(f"List of available Plugins for the Planner: {list(kernel.skills.data.keys())}")

Now that we know the list of Plugins available, let's create our plan for a ASK that can be solved with one of them.

In [ ]:
#We need to inform what we need to achieve to the planner define which Plugins (and Functions it'll pick)
ask = "What is the time right now?"

try:
    #Now we'll ask our planner to create a plan to achieve the goal we defined
    plan = await planner.create_plan_async(goal=ask)

    #And execute the plan
    result = await plan.invoke_async()
    print(f"Result: {result}")

except Exception as e:
     print(f"Error: {e}")

The ActionPlanner picked a single Plugin (and function) to get back to us with a Response based on our ask. Eventually, our ask may require Plugins that are not available (or were not loaded) in the kernel. Let's see an example. 

In [ ]:
#Let's see which Plugins are available for the Planner
print(f"List of available Plugins for the Planner: {list(kernel.skills.data.keys())}")

#Defining a different ask for a Plugin that is not available
ask = "Translate the following text to Spanish: I love Semantic Kernel"

try:
    #Creating and executing the Plan
    plan = await planner.create_plan_async(goal=ask)
    result = await plan.invoke_async()
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")

As expected, we got an error / no result as we haven't loaded any Plugins that are capable of translations. Let's import a new Plugin into the Kernel and see how we can list the functoins available for the Planner.

In [ ]:
#Define where the plugins are stored. If your plugins are in a different directory, change this parameter.
sample_plugins_directory = "./sksampleplugins"

#Adding the WriterPlugin to the kernel, which has a function to translate text
writer_plugin = kernel.import_semantic_skill_from_directory(sample_plugins_directory, "WriterPlugin")

#Let's see which Plugins and  available now and their fuctions
for plugin_name, functions in kernel.skills.data.items():  
    print(f"Plugin: {plugin_name}")  
    for function_name in functions.keys():  
        print(f"  Function: {function_name}") 

Finally, as we now have the proper Plugin, let's see how the same "ASK" is handled.

In [ ]:
#Now that we have the WriterPlugin imported, we can use its functions
ask = "Translate the following text to Spanish: I love Semantic Kernel"

try:
    #Creating and executing the Plan
    plan = await planner.create_plan_async(goal=ask)
    result = await plan.invoke_async()
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")

#### Step 3 - Basic Planner
We saw how the Action Planner can help to find a single Plugin to achieve a goal, however, most of the times, the real power is to combine several Plugins in a given order to achieve a goal, which is the role of **Sequential Planner** and **Basic Planner**. Since they're similar, we'll explore only Basic Planner (later we'll see Sequential Planner in another lab).

The Basic Planner will produce a plan (in JSON format) stating the steps (i.e, functions inside Plugins) it'll use to achieve the goal. Each step is run and evaluated sequentially.

Let's how it works.

In [ ]:
from semantic_kernel.planning import BasicPlanner

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

#Import the Planner from its module. Planners are part of "planning" module
from semantic_kernel.planning import ActionPlanner

# Initialize the kernel
kernel = sk.Kernel()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env() 

#Adding a model to the kernel
kernel.add_chat_service("Chat Completion",
                        AzureChatCompletion(deployment, endpoint, api_key))

#Define where the plugins are stored. If your plugins are in a different directory, change this parameter.
sample_plugins_directory = "./sksampleplugins"

#Defining a single Plugin with several functions to be used by the Planner
writer_plugin = kernel.import_semantic_skill_from_directory(sample_plugins_directory, "WriterPlugin")

#Define the Planner
planner = BasicPlanner()

#Define the goal for the planner
ask = """Generate taglines for a telco company for a marketing campaign. 
Translate each of the ideias to Spanish as bullet points and send as an email to Marketing. 
"""
#Let's see which Plugins are available now
print(f"List of available Plugins for the Planner: {list(kernel.skills.data.keys())}")

#Print the steps of the plan
try:
    plan = await planner.create_plan_async(goal=ask, kernel=kernel)
    print(f"Plan: {plan.generated_plan}")
except Exception as e:
    print(f"Error: {e}")


The Basic Planner used the LLM to produce a plan on how to achieve the goal given user's ask. The plan is a sequence of subtasks each represented by a function among loaded Plugins. The input is extracted from user's ask and the otuput of each step is the input for the next one.

Also notice that the plan has also determined additional parameters for the functions when required.

Eventually, the plan may not look like as expected, so run previous cell to ensure the plan looks accurate for the given ask.

 If the plan is correct, let's execute it.

In [ ]:
#Try to execute the plan
try:
    result = await planner.execute_plan_async(plan, kernel)
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")

#### Step 4 - Stepwise Planner to orchestrate a Call Center Analytics solution

In challenging scenarios, the previous planners may not be able to achieve user's goal in a sequential way. One alternative for planning is to ask the LLM to propose a high-level plan, execute each step and **reason** if the output helps or not to achieve user's goal, adjusting the plan accordingly until the goal is met or the maximum if iterations is reached. 

StepWise Planner does exactly that, inspired on advanced techniques, like [**Modular Reasoning, Knowledge and Language - MRKL**](https://arxiv.org/abs/2210.03629) and [**ReACT pattern**](https://arxiv.org/abs/2210.03629). For each step, this planner will produce:
- **Thoughts:** what the LLM believe, in natural language, it's required to do Next to be closer to achieve user's goal;
- **Actions:** what kind of action (i.e, Functions from a Plugin with its parameters) they will do next to achieve the THOUGHT.
- **Observations:** outputs produced from the ACTION that can be a final answer or the input for the next THOUGHT.

Each step should help the plan to com closer to user's goal. However, sometimes the planner may fail to converge to a solution. Therefore, it's possible to control the maximum number of interactions the plan it'll take. Also note that each step is a request to the LLM. Therefore, it's important to think on costs when using this type of planner. Moreover, planners in general can be unpredictable sometimes, so always evaluate well before using them for critical scenarios in production.

Now, let's see how we can use this Planner to run our **Call Center Analytics Scenario** in a more autonomous way. Our goal is to run all the steps below from a single "ask" in natural language using StepWise Planner.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

#Importing modules required for this Planner
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

#Importing the native function within the Plugin
from plugins.CallCenterPlugin.Transcribe import Transcribe

#Importing required modules to load sensitive data from .env file
from dotenv import load_dotenv 
import os

#Load sensitive data from .env file to use Azure AI Speech
load_dotenv()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

#Reloading kernel and chat servince to ensure it's accurate
kernel = sk.Kernel()

#Adding a LLM to be used by the kernel
kernel.add_chat_service("Chat Completion",
                        AzureChatCompletion(deployment, endpoint, api_key))

#Define where the plugins are stored. If your plugins are in a different directory, change this parameter.
plugins_directory = "./plugins"
sample_plugins_directory = "./sksampleplugins"

#Now, we'll add the Plugins that the kernel will be allowed to use. 
# In this case, we're importing core native functions from Call Center Plugin and semantic functions from Writer Plugin and Call Center Plugin 
kernel.import_skill(Transcribe(), "CallCenterPlugin")
callcenter_plugin = kernel.import_semantic_skill_from_directory(plugins_directory, "CallCenterPlugin")
writer_plugin = kernel.import_semantic_skill_from_directory(sample_plugins_directory, "WriterPlugin")

#Let's confirm which Plugins are available for the Planner use
print(f"List of available Plugins for the Planner: {list(kernel.skills.data.keys())}")

#Defining the input variables for the transcription funcion
transcription_param = {}
transcription_param["input"] = "./audio/english_billing_process_sample.wav" #"./audio/enUS_short_wifiproblem.wav"
transcription_param["language"] = "en-US"
transcription_param["subscription_key"] = os.getenv("AZURE_AI_SPEECH_KEY")
transcription_param["region"] = os.getenv("AZURE_AI_SPEECH_REGION")

#Defining the ask in a single statement 
ask = f"""
Transcribe the audio file, summarize the incident and extract the category of the problem as Mobile Internet, Fixed Internet, Wifi/Router or Billing.
Write an apology email to Customer demonstrating that you understood their problem. Be brief in the email. 
Audio transcription details: "{transcription_param}"
"""

#Create the plan already defining the max number of iterations and how fast each interaction should be  
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

#Try to create the plan
try:
    plan = planner.create_plan(goal=ask)  
except Exception as e:
    print(f"Error: {e}")

Leet's execute the plan and see it's response. It can take sometime depending on the type of functions used and the number of interactions that the plan take to converge to a solution. Remember that each step is a call to the LLM, so Costs are also an important consideration.

In [ ]:
#Trying to execute the plan. It can take sometime due to the transcription duration and the number of steps expected in the plan
try:
    result = await plan.invoke_async()
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")

Now we can review what were the steps that the plan took to seek user's goal. Please note Thoughts, Actions and Observations as intermediary steps towards a solution. Since Stepwise Planner builds the plan in runtime, as it executes actions and reason over each observation, we can see the full plan after the execution.

In [ ]:
#Listing steps taken by the plan
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))